In [ ]:
pip install -r requirements.txt --quiet

In [ ]:
# perform some setup

from google.cloud import aiplatform
from google.cloud import storage
import joblib
import numpy as np
import pandas as pd


PROJECT = "jwd-test-sbcl"
LOCATION = "us-central1"

MODEL_NAME = "adult-income-ap-training-model"
BUCKET_NAME = "jwd-test-sbcl"
GCS_FOLDER = "adult-income-ap-training-model"

JOB_NAME = "adult-income-ap-training-job"
MACHINE_TYPE = "n2-highmem-2"
REPLICA_COUNT = 1
EXECUTOR_IMAGE_URI = "us-docker.pkg.dev/vertex-ai/training/tf-cpu.2-13.py310:latest"
WORKING_DIRECTORY = "."
SCRIPT_PATH = "train.py"



In [ ]:
# use the gcloud CLI to create a custom training job using auto packagin

! gcloud ai custom-jobs create \
  --region=$LOCATION \
  --display-name=$JOB_NAME \
  --worker-pool-spec=machine-type=$MACHINE_TYPE,replica-count=$REPLICA_COUNT,executor-image-uri=$EXECUTOR_IMAGE_URI,local-package-path=$WORKING_DIRECTORY,script=$SCRIPT_PATH

In [ ]:
# initialize aiplatform
aiplatform.init(project=PROJECT, location=LOCATION)

# Create the model
model = aiplatform.Model.upload(
    display_name=MODEL_NAME,
    artifact_uri=f'gs://{BUCKET_NAME}/{GCS_FOLDER}',
    serving_container_image_uri='us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-13:latest',
)

print(f'Model ID: {model.resource_name}')

# Deploy the model to the endpoint
endpoint = model.deploy(
    deployed_model_display_name=f'{MODEL_NAME}-deployed',
    machine_type='n2-highmem-2'
)

print(f'Endpoint ID: {endpoint.resource_name}')

In [ ]:
# copy the pickled preprocessing stuff
storage_client = storage.Client()
bucket = storage_client.bucket(BUCKET_NAME)
pkls = [
    "scaler.pkl",
    "label_encoder.pkl",
    "categorical_encoder.pkl"
]

for pkl in pkls:
    blob = bucket.blob(f"{GCS_FOLDER}/{pkl}")
    blob.download_to_filename(pkl)

# load the pre-trained ML preprocessing objects (scaler, label_encoder, and categorical_encoder)
scaler = joblib.load('scaler.pkl')
label_encoder = joblib.load('label_encoder.pkl')
categorical_encoder = joblib.load('categorical_encoder.pkl')


In [ ]:
# populate the list of instances for prediction
instances = [
    [39,"Private", "9th",5,"Married-civ-spouse","Other-service","Wife","Black","Female",3411,0,34,"United-States"],
    [77,"Private", "9th",5,"Married-civ-spouse","Priv-house-serv","Wife","Black","Female",0,0,10,"United-States"],
    [27,"Local-gov","HS-grad",9,"Married-civ-spouse","Exec-managerial","Husband","White","Male",0,0,80,"United-States"],
    [40,"Private","Masters",14,"Married-civ-spouse","Exec-managerial","Husband","White","Male",0,0,46,"United-States"]

]

# what is this code doing? why does it work?
instances_numeric_features = np.array([[instance[0], instance[3], instance[9], instance[10], instance[11]] for instance in instances])
instances_scaled_numeric_features = scaler.transform(instances_numeric_features)

# what is this code doing? why does it work?
instances_categorical_features = np.array([instance[1:3] + instance[4:9] + [instance[12]] for instance in instances])
instances_categorical_encoded = categorical_encoder.transform(instances_categorical_features)

# what is this code doing?
instances_combined = np.hstack((instances_scaled_numeric_features, instances_categorical_encoded))

# what is this code doing? why is it important?
preprocessed_instances_as_list = instances_combined.tolist()

# call the predict method and pass the instances for prediction
response = endpoint.predict(instances=preprocessed_instances_as_list)

# what is response.predictions?
# what is a prediction?
# what is argmax doing?
# what is inverse_transform doing?
for prediction in response.predictions:
    predicted_label = np.argmax(prediction)
    decoded_label = label_encoder.inverse_transform([predicted_label])[0]
    print(f'Predicted Label: {decoded_label}')
    print(f'Predicted Probabilities: {prediction}')